In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import ee
import geemap
import geetools
from datetime import datetime

In [2]:
ee.Initialize(project='ee-jonstar')

In [3]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

## CONUS Geometry

In [4]:
conus = ee.ImageCollection("GRIDMET/DROUGHT").filterDate('2000-01-01', '2001-01-01')
geometry = conus.geometry();

In [5]:
# Define point near Baltimore
u_lon = -76.6122
u_lat = 39.2904
u_poi = ee.Geometry.Point(u_lon, u_lat)
roi = u_poi.buffer(2e2)

# Point for Baltimore SAR image
lon1 = -78.08
lat1 = 39.417
pt1 = ee.Geometry.Point(lon1, lat1)

# Point for Washington SAR image
lon2 = -77.723
lat2 = 38.686
pt2 = ee.Geometry.Point(lon2, lat2)

## Landsat Initialization

In [6]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
landsat_US = landsat.filterDate('2023-01-01', '2023-12-31').filterBounds(roi)

In [7]:
landsat_US.first().projection()

In [8]:
lList = landsat_US.toList(8)

In [9]:
# List of days Landsat should have an image, noting missing ones (for 2024, though I am switching to 2023 data)
# January 14, 2024
# January 30, 2024 - MISSING
# February 15, 2024
# March 2, 2024 - MISSING
# March 18, 2024
# April 3, 2024 - MISSING
# April 19, 2024 - MISSING
# May 5, 2024 - MISSING
# May 21, 2024
# June 6, 2024
# June 22, 2024
# July 8, 2024
# July 24, 2024 - MISSING
# August 9, 2024 - MISSING
# August 25, 2024

ee.Image(lList.get(0)).date()

## GOES Initialization

In [10]:
# Landsat images over Baltimore regularly taken around 15:47 every 16 days
hour_filt1 = ee.Filter.calendarRange(15, 15, 'hour')
minute_filt = ee.Filter.calendarRange(45, 49, 'minute')
time_filt1 = ee.Filter.And(hour_filt1, minute_filt)

hour_filt2 = ee.Filter.calendarRange(3, 3, 'hour')
time_filt2 = ee.Filter.And(hour_filt2, minute_filt)

time_filt = ee.Filter.Or(time_filt1, time_filt2)

In [11]:
GOES = ee.ImageCollection("NOAA/GOES/16/MCMIPC").filterDate('2023-01-01', '2023-12-31')\
                .filter(time_filt)

In [12]:
gList = GOES.toList(15)

In [13]:
ee.Image(gList.get(12)).date()

## Sentinel-1 SAR Initialization

In [14]:
pt1_filt = ee.Filter.bounds(pt1)
pt2_filt = ee.Filter.bounds(pt2)

pts_filt = ee.Filter.Or(pt1_filt, pt2_filt)

In [15]:
sentinel = ee.ImageCollection("COPERNICUS/S1_GRD").filterDate('2023-01-01', '2023-12-31').filter(pts_filt)

In [16]:
print(sentinel.size().getInfo())

60


In [17]:
#slist = sentinel.toList(2)

In [18]:
#ee.Image(slist.get(1)).date()

## Filtering images by time

In [19]:
#join = ee.Join.inner()
#all_join = ee.Join.saveAll(matchKey='matched_img', measureKey='time_diff', outer=True)
best_join = ee.Join.saveBest(matchKey='matched_img', measureKey='time_diff', outer=True)
# Difference value is equal to 8 days and 5 minutes in milliseconds
timeFiltLandsatGOES = ee.Filter.maxDifference(difference=691500e3, leftField='system:time_start', rightField='system:time_start')

timeFiltLandsatSentinel = ee.Filter.maxDifference(difference=518400e4, leftField='system:time_start', rightField='system:time_start')

In [20]:
col_near = ee.ImageCollection(best_join.apply(GOES, landsat_US, timeFiltLandsatGOES))
LandsatSentinel_near = ee.ImageCollection(best_join.apply(landsat_US, sentinel, timeFiltLandsatSentinel))

In [21]:
col_near.size().getInfo()

725

In [22]:
landsat_US.size().getInfo()

20

In [23]:
GOES.size().getInfo()

725

In [24]:
LandsatSentinel_near.size().getInfo()

20

In [25]:
im = col_near.first()

In [26]:
im.get('matched_img')

## Helper Functions

In [27]:
# Applies scale and offset factors for Landsat band 10 LST
def scale_and_offset_LST(image):
    return image.select('ST_B10').multiply(0.00341802).add(149)

# Applies scale and offset factors for Landsat reflectance bands
def scale_and_offset_Landsat(image):
    return image.select(['SR_B2', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']).multiply(2.75e-05).add(-0.2)

In [28]:
# Applies scale and offset factors for GOES imagery
# All scale and offset values were taken straight from GEE to save compute
def scale_and_offset_GOES(image):
    reflectances = image.select(['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05', 'CMI_C06']).multiply(0.0002442)
    brightness_temps = image.select(['CMI_C14', 'CMI_C15']).multiply(0.039316241).add(173.14999)
    
    return reflectances.addBands(srcImg=brightness_temps, names=['CMI_C14', 'CMI_C15'])

In [29]:
# Preprocessing function
def process_GOES(image):
    #####################################
    # Landsat portion
    
    # Scaling and offset
    #image1 = ee.Image(feature.get('primary'))
    #landsat_LST = scale_and_offset_LST(feature)
    #landsat_image = scale_and_offset_Landsat(feature)
    # Add cloud mask back in
    #landsat_image = landsat_image.addBands(srcImg=landsat_LST, names=['ST_B10'])\
    #                            .addBands(srcImg=feature, names=['QA_PIXEL'])

    ######################################
    # GOES portion

    # Scaling and offset
    #image2 = ee.Image(feature.get('matched_img'))
    GOES_image = scale_and_offset_GOES(image)

    ######################################
    # Timestamp portion
    
    # Function to get time from GOES image\    
    def get_GOES_time(f):
        GOES_time = ee.Number(image.get('system:time_start'))       
        return f.set('timestamp', GOES_time)

    # Add timestamp back in
    #landsat_image = get_GOES_time(landsat_image)
    GOES_image = get_GOES_time(GOES_image)

    ##########################################
    
    #image = landsat_image.addBands(srcImg=GOES_image, names=['CMI_C01', 'CMI_C02', 'CMI_C03', 'CMI_C05',
    #                                                         'CMI_C06', 'CMI_C14', 'CMI_C15']).cast({'QA_PIXEL':'double'})
    return GOES_image

In [30]:
# Preprocessing function
def process_Landsat_Sentinel(feature):
    ######################################
    # Sentinel portion

    # Scaling and offset
    sentinel_image = ee.Image(feature.get('matched_img'))
    
    #####################################
    # Landsat portion
    
    # Scaling and offset
    feature = ee.Image(feature)
    #image1 = ee.Image(feature.get('primary'))
    landsat_LST = scale_and_offset_LST(feature)
    landsat_image = scale_and_offset_Landsat(feature)
    # Add cloud mask back in
    landsat_image = landsat_image.addBands(srcImg=landsat_LST, names=['ST_B10'])\
                                .addBands(srcImg=feature, names=['QA_PIXEL'])

    ######################################
    # Timestamp portion
    
    # Function to get time from Landsat image
    def get_Landsat_time(f):
        Landsat_time = ee.Number(feature.get('system:time_start'))       
        return f.set('timestamp', Landsat_time)

    # Add timestamp back in
    landsat_image = get_Landsat_time(landsat_image)

    ##########################################
    
    image = landsat_image.addBands(srcImg=sentinel_image).cast({'QA_PIXEL':'double', 'angle':'double'})
    
    return image

## Produce Landsat/GOES geotiffs and their respective times

In [31]:
processed = GOES.map(process_GOES)

In [32]:
first = processed.first()

In [33]:
first

In [34]:
times = processed.aggregate_array('timestamp')

In [35]:
def times_to_features(num):
    return ee.Feature(None, {'value': num})

features = ee.FeatureCollection(times.map(times_to_features))

In [138]:
task = ee.batch.Export.table.toDrive(
    collection=features,
    description='GOES_times',
    folder='GOES_Landsat_SAR_DMV',
    fileFormat='CSV',
)
task.start()

## Produce Landsat/Sentinel geotiffs and their respective times

In [36]:
processed2 = LandsatSentinel_near.map(process_Landsat_Sentinel)

In [37]:
first = ee.Image(processed2.toList(13).get(12))

In [38]:
first

In [39]:
times2 = processed2.aggregate_array('timestamp')
features2 = ee.FeatureCollection(times2.map(times_to_features))

In [120]:
task = ee.batch.Export.table.toDrive(
    collection=features2,
    description='Landsat_times',
    folder='GOES_Landsat_SAR_DMV',
    fileFormat='CSV',
)
task.start()

## Export stuff

In [40]:
# Export images, only taking area over Baltimore and Washington
num = processed.size().getInfo()
#num = 1
img_list = processed.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'GOES_image_{i}', fileFormat='GeoTIFF', folder='GOES_Landsat_SAR_DMV',
                dimensions='3000x3000', crs='EPSG:32618', crsTransform=[30, 0, 292000, 0, -30, 4372200])
    task.start()

In [68]:
# Export images, only taking area over Baltimore and Washington
#num = processed2.size().getInfo()
num = 1
img_list = processed2.toList(num)

for i in list(range(num)):
    img = ee.Image(img_list.get(i))
    
    task = ee.batch.Export.image.toDrive(
        img, description=f'Landsat_Sentinel_image_{i}', fileFormat='GeoTIFF', folder='GOES_Landsat_SAR_DMV',
                dimensions='3000x3000', crs='EPSG:32618', crsTransform=[30, 0, 292000, 0, -30, 4372200])
    task.start()

## Try from gee code

In [28]:
geo = sample.select('ST_B10').geometry()

In [29]:
Map = geemap.Map(center=[u_lat, u_lon], zoom=4)
Map.add_basemap("HYBRID")

Map.addLayer(sample.select('ST_B10'), {'min':250, 'max':325,
    'palette':['#fef0d9', '#fdcc8a', '#fc8d59', '#e34a33', '#b30000']}, 'LST')
Map.addLayer(sample.select('CMI_C14').clip(geo), {'min':96, 'max':342,
    'palette':['#f7f7f7', '#cccccc', '#969696', '#636363', '#252525']}, 'GOES14')

In [30]:
Map

Map(center=[39.2904, -76.6122], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…